# Modelos mecanicistas para a hidrólise enzimática

As equações a seguir representam subreações da etapa de hidrólise enzimática de matéria prima lignocelulósica.

## Variáveis

In [42]:
# Ef = 1
# Eb = 1
# S = 1
# S0 = 1 
# C = 1
# L = 1
# H = 1
# G = 1
# G2 = 1
# X = 1

## Parâmetros ajustados

In [43]:
Emax = 8.32
Kad = 7.16
alpha = 1
k1r = 0.177
k2r = 8.81
k3r = 201.0
k4r = 16.34
K1iG = 0.402
K1iG2 = 2.71
K1iX = 2.15
K2iG = 119.6
K2iG2 = 4.69
K2iX = 0.095
K3M = 26.6
K3iG = 11.06
K3iX = 1.023
K4iG = 16.25
K4iG2 = 4.0
K4iX = 154.0

## Equações

### Definições

In [44]:
#Langmuir-type isotherm 1

def Ebs1(Eb , S):

    Ebs1 = Eb/S

    return Ebs1

In [45]:
# Langmuir-type isotherm 2

def Ebs2(Emax, Kad, Ef):

    Ebs2 = Emax*Kad*Ef/(1+Kad*Ef)

    return Ebs2

In [46]:
# Enzyme adsorbed on cellulose

def Ebc(Eb, C, S):

    Ebc = Eb*C/S

    return Ebc

In [47]:
# Enzyme adsorbed on hemicellulose
def Ebh(Eb, H, S):

    Ebh = Eb*H/S

    return Ebh

In [48]:
# Substrate reactivity

def Rs(alpha, S, S0):

    Rs = alpha*S/S0

    return Rs

### Taxas de reação

In [49]:
# Reaction 1: Cellulose to cellobiose

def r1(k1r, Ebc, S, Rs, G2, G, X, K1iG, K1iG2, K1iX):

    r1 = k1r*Ebc*S*Rs/(1+ G2/K1iG2 + G/K1iG + X/K1iX)
    return r1

In [50]:
# Reaction 2: Cellulose to glucose
def r2(k2r, Ebc, S, Rs, G2, G, X, K2iG, K2iG2, K2iX):

    r2 = k2r*Ebc*S*Rs/(1+ G2/K2iG2 + G/K2iG + X/K2iX)
    return r2

In [51]:
# Reaction 3: Cellobiose to glucose
def r3(k3r, Ef, G2, G, X, K3M, K3iG, K3iX):

    r3 = k3r*Ef*G2/(((1 + G/K3iG + X/K3iX)*K3M)+G2)

    return r3

In [52]:
# Reaction 4: Hemicellulose to Xilose
def r4(k4r, Ebh, Rs, S, G2, G, X, K4iG, K4iG2, K4iX):

    r4 = k4r*Ebh*Rs*S/(1 + G2/K4iG2 + G/K4iG + X/K4iX)

    return r4

### Balanços de massa

In [53]:
# Total Enzyme
def Et(Eb, Ef):
    Et = Eb + Ef
    return Et

In [54]:
# Total Solids

def S(C, H, L):
    S = C + H + L
    return S

### Taxa de variação de componentes

In [55]:
# Cellulose balance

def dC_dt(r1, r2):

    dC_dt = -r1 - r2
    return dC_dt

In [56]:
# Cellobiose balance

def dG2_dt(r1, r3):
    dG2_dt = 1.056*r1 - r3
    return dG2_dt

In [57]:
# Glucose balance
def dG_dt(r2, r3):
    dG_dt = 1.111*r2 + 1.053*r3
    return dG_dt

In [58]:
# Hemicellulose balance

def dH_dt(r4):
    dH_dt = -r4
    return dH_dt

In [59]:
# Xylose balance
def dX_dt(r4):
    dX_dt = 1.136*r4
    return dX_dt

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

# Defina a composição da biomassa e a carga de sólidos (exemplo)
biomass_composition = {
    'cellulose': 0.66,        # fração mássica de celulose
    'hemicellulose': 0.083,   # fração mássica de hemicelulose
    'lignin': 0.257           # fração mássica de lignina
}
solids_loading_g_L = 15  # concentração total de sólidos em g/L

# Calculando as concentrações iniciais
C0 = solids_loading_g_L * biomass_composition['cellulose']
H0 = solids_loading_g_L * biomass_composition['hemicellulose']
L0 = solids_loading_g_L * biomass_composition['lignin']

S0 = C0 + H0 + L0  # S0 calculado corretamente

# Defina Ef0 e Eb0
Ef0 = 10  # g/L
Eb0 = 0   # g/L

# Função que define o sistema de equações diferenciais
def hydrolysis_ode(t, y, params, S0=S0):
    C, G2, G, H, X, Eb, Ef = y  # Agora Eb e Ef são variáveis de estado

    # Parâmetros do dicionário (apenas os cinéticos)
    alpha = params["alpha"]
    Emax = params["Emax"]
    Kad = params["Kad"]
    k1r = params["k1r"]
    k2r = params["k2r"]
    k3r = params["k3r"]
    k4r = params["k4r"]
    K1iG = params["K1iG"]
    K1iG2 = params["K1iG2"]
    K1iX = params["K1iX"]
    K2iG = params["K2iG"]
    K2iG2 = params["K2iG2"]
    K2iX = params["K2iX"]
    K3M = params["K3M"]
    K3iG = params["K3iG"]
    K3iX = params["K3iX"]
    K4iG = params["K4iG"]
    K4iG2 = params["K4iG2"]
    K4iX = params["K4iX"]

    # Variáveis auxiliares
    S = C + H + L0
    Rs = alpha * S / S0
    Ebc = Eb * C / S if S > 0 else 0
    Ebh = Eb * H / S if S > 0 else 0

    # Taxas de reação
    r1 = k1r * Ebc * S * Rs / (1 + G2 / K1iG2 + G / K1iG + X / K1iX)
    r2 = k2r * Ebc * S * Rs / (1 + G2 / K2iG2 + G / K2iG + X / K2iX)
    r3 = k3r * Ef * G2 / (((1 + G / K3iG + X / K3iX) * K3M) + G2)
    r4 = k4r * Ebh * Rs * S / (1 + G2 / K4iG2 + G / K4iG + X / K4iX)

    # Adsorção/dessorção enzimática (Langmuir)
    taxa_ads = Kad * Ef * (Emax - Eb)
    taxa_dess = Kad * Eb
    dEb_dt = taxa_ads - taxa_dess
    dEf_dt = -dEb_dt

    # Equações diferenciais
    dC_dt = -r1 - r2
    dG2_dt = 1.056 * r1 - r3
    dG_dt = 1.111 * r2 + 1.053 * r3
    dH_dt = -r4
    dX_dt = 1.136 * r4

    return [dC_dt, dG2_dt, dG_dt, dH_dt, dX_dt, dEb_dt, dEf_dt]

# Parâmetros do modelo como dicionário (apenas os cinéticos)
params = {
    "alpha": 1,
    "Emax": 8.32,
    "Kad": 7.16,
    "k1r": 0.177,
    "k2r": 8.81,
    "k3r": 201.0,
    "k4r": 16.34,
    "K1iG": 0.402,
    "K1iG2": 2.71,
    "K1iX": 2.15,
    "K2iG": 119.6,
    "K2iG2": 4.69,
    "K2iX": 0.095,
    "K3M": 26.6,
    "K3iG": 11.06,
    "K3iX": 1.023,
    "K4iG": 16.25,
    "K4iG2": 4.0,
    "K4iX": 154.0
}
# Condições iniciais
y0 = [C0, 0, 0, H0, 0, Eb0, Ef0] # C, G2, G, H, X, Eb, Ef

# Intervalo de tempo para a simulação
t_span = (0, 96)  # De 0 a 96 horas
t_eval = np.linspace(t_span[0], t_span[1], 200)  # Pontos de avaliação

solution = solve_ivp(
    hydrolysis_ode, t_span, y0, args=(params, S0), t_eval=t_eval
)

# Eb e Ef ao longo do tempo
Eb_t = solution.y[5]
Ef_t = solution.y[6]

# Calcular S(t) ao longo do tempo
C = solution.y[0]
H = solution.y[3]
L = L0  # Lignina é constante
S_t = C + H + L

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 7))
plt.plot(solution.t, C, label="C (Cellulose)")
plt.plot(solution.t, solution.y[1], label="G2 (Cellobiose)")
plt.plot(solution.t, solution.y[2], label="G (Glucose)")
plt.plot(solution.t, H, label="H (Hemicellulose)")
plt.plot(solution.t, solution.y[4], label="X (Xylose)")
plt.plot(solution.t, S_t, '--', label="S (Total solids)")
plt.xlabel("Time (h)")
plt.ylabel("Concentration (g/L)")
plt.legend()
plt.title("Simulation of Enzymatic Hydrolysis")
plt.grid()
plt.show()

NameError: name 'dEb_dt' is not defined

In [ ]:
import pandas as pd

# Gerar o dataframe com os resultados da simulação
def generate_dataframe(solution):
    data = {
        "Time": solution.t,
        "Cellulose (C)": solution.y[0],
        "Cellobiose (G2)": solution.y[1],
        "Glucose (G)": solution.y[2],
        "Hemicellulose (H)": solution.y[3],
        "Xylose (X)": solution.y[4]
    }
    df = pd.DataFrame(data)
    return df

# Criar o dataframe
df_simulation = generate_dataframe(solution)

# Baixar o dataframe como CSV
# df_simulation.to_csv("enzymatic_hydrolysis_simulation.csv", index=False)

# Exibir os pontos
df_simulation

,Time,Cellulose (C),Cellobiose (G2),Glucose (G),Hemicellulose (H),Xylose (X)
0,0.000000,9.900000,0.000000e+00,0.000000,1.245000e+00,0.000000
1,0.482412,7.149201,3.931966e-03,3.052096,4.516741e-03,1.409189
2,0.964824,5.913205,2.170878e-03,4.427175,5.648451e-05,1.414256
3,1.447236,4.993784,1.418101e-03,5.449465,1.380183e-06,1.414318
4,1.929648,4.282227,1.009642e-03,6.240448,5.342168e-08,1.414320
...,...,...,...,...,...,...
195,94.070352,0.000004,-1.313606e-06,10.999121,1.022724e-114,1.414320
196,94.552764,0.000003,-1.704489e-07,10.999120,3.075713e-115,1.414320
197,95.035176,0.000003,1.175153e-06,10.999119,9.249833e-116,1.414320
198,95.517588,0.000003,6.350222e-07,10.999120,2.781764e-116,1.414320
